<a href="https://colab.research.google.com/github/UtkarshAIITB/IPL-Data-Analysis/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/new.csv')
# new.head()

In [4]:
t = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/t.csv')
# t.head()

In [5]:
t.drop(columns = ['Unnamed: 0'], inplace = True)
t.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,597998,2013-04-03,9,6,9,9,1,0.1,6,9,2013,1,0,0,128,0,1
1,597998,2013-04-03,9,6,9,9,1,0.2,6,9,2013,1,0,0,128,4,1
2,597998,2013-04-03,9,6,9,9,1,0.3,6,9,2013,1,0,0,128,5,1
3,597998,2013-04-03,9,6,9,9,1,0.4,6,9,2013,1,0,0,128,5,1
4,597998,2013-04-03,9,6,9,9,1,0.5,6,9,2013,1,0,0,128,5,1


In [6]:
new.drop(columns = ['Unnamed: 0'], inplace = True)

In [7]:
new.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,9,5,2008,1,0,0,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,9,5,2008,1,0,0,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,9,5,2008,1,0,0,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,9,5,2008,1,0,0,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,9,5,2008,1,0,0,222,2,0


In [8]:
# new is the training set and t is the test set

In [9]:
tnew = t.drop(t[(t.over <= 11.0) | (t.over >=16.0)].index)
tnew.reset_index(drop = True)
y1test = tnew['total_runs_y']
y2test = tnew['winner']
tnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
tnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,9,6,9,1,11.1,6,9,1,0,0,76,3
1,9,6,9,1,11.2,6,9,1,0,0,77,3
2,9,6,9,1,11.3,6,9,1,0,0,78,3
3,9,6,9,1,11.4,6,9,1,0,0,79,3
4,9,6,9,1,11.5,6,9,1,0,0,79,3


In [10]:
xnew = new.drop(new[(new.over <=11.0) | (new.over >=16.0)].index)
xnew.reset_index(drop = True)
y1train = xnew['total_runs_y']
y2train = xnew['winner']
xnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
xnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,11.1,9,5,1,0,0,102,1
1,5,9,5,1,11.2,9,5,1,0,0,108,1
2,5,9,5,1,11.3,9,5,1,0,0,110,1
3,5,9,5,1,11.4,9,5,1,0,0,111,1
4,5,9,5,1,11.5,9,5,1,0,0,111,1


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [12]:
xnewt = sc.fit_transform(xnew) 
tnewt = sc.fit_transform(tnew)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [84]:
model = Sequential()
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(100, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(50, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [85]:
model.fit(x=xnewt, y=y1train, epochs=200, 
          validation_data=(tnewt,y1test))

Epoch 1/200
1370/1370 [==============================] - 3s 2ms/step - loss: 3049.3257 - val_loss: 346.1981
Epoch 2/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1686.5195 - val_loss: 269.6570
Epoch 3/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1596.0295 - val_loss: 262.4864
Epoch 4/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1529.0812 - val_loss: 288.1367
Epoch 5/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1497.9878 - val_loss: 296.3682
Epoch 6/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1435.5721 - val_loss: 267.1111
Epoch 7/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1391.7900 - val_loss: 257.8290
Epoch 8/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1364.9725 - val_loss: 272.6342
Epoch 9/200
1370/1370 [==============================] - 3s 2ms/step - loss: 1340.5673 - val_loss: 262.5283
Epoch 10/200
1370/1370 [====

In [86]:
y1test = y1test.ravel()

In [87]:
predictions = model.predict(tnewt)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=y1test
sample.head(10)

,Predict,Actual
0,164.682068,128
1,165.171799,128
2,165.717056,128
3,166.267487,128
4,165.613586,128
5,141.648499,128
6,141.779663,128
7,141.419312,128
8,141.323044,128
9,146.508286,128


In [88]:
def custom_accuracy(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

In [89]:
custom_accuracy(sample['Actual'] , sample['Predict'],10)

42.08916083916084

In [90]:
# sample.to_csv('sample.csv')

In [91]:
# try1 = sample[sample.Actual == 128]
# try1.head()

In [92]:
# try1.shape[0]

In [93]:
# try1.sum()

In [94]:
# (try1.sum())/try1.shape[0]

In [95]:
# try2 = sample[sample.Actual == 129]
# (try2.sum())/try2.shape[0]

In [96]:
# try2 = sample[sample.Actual == 156]
# result = (try2.sum())/try2.shape[0]

In [97]:
# result

In [98]:
# res2 = result.values.reshape(1,2)

In [99]:
# res2

In [100]:
# res2.shape

In [101]:
# df.loc[len(df)] = your_array
# results.loc[len(results)] = res2
# df.append(pd.DataFrame(arr).T)
# results = results.append(pd.DataFrame(res2))
# results.append(pd.DataFrame(res3))

In [102]:
ar1 = sample['Actual'].unique()

In [103]:
results = pd.DataFrame()

In [104]:
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [105]:
results.head()

,0,1
0,144.348239,128.0
0,148.233099,129.0
0,153.139252,156.0
0,160.135460,154.0
0,144.348145,126.0


In [106]:
# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)

In [107]:
results.reset_index(drop = True)

,Predict,Actual
0,144.348239,128.0
1,148.233099,129.0
2,153.139252,156.0
3,160.135460,154.0
4,144.348145,126.0
...,...,...
79,196.724869,190.0
80,128.490400,113.0
81,162.502268,172.0
82,147.217182,134.0


In [108]:
results.tail()

,Predict,Actual
0,196.724869,190.0
0,128.490400,113.0
0,162.502268,172.0
0,147.217182,134.0
0,170.402539,192.0


In [109]:
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

48.80952380952381


In [110]:
# results.info()

In [111]:
# results['Actual']

take epochs = 90 next time

In [112]:
tests = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/testset.csv')
# tests.head()
tests.drop(columns = ['Unnamed: 0'], inplace = True)
tests.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.1,12.0,5.0,1,0,0,159,2,0
1,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.2,12.0,5.0,1,0,0,159,2,0
2,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.3,12.0,5.0,1,0,0,159,2,0
3,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.4,12.0,5.0,1,0,0,159,4,0
4,1254058,2021-04-09,12.0,5.0,5.0,5.0,1,0.5,12.0,5.0,1,0,0,159,4,0


In [113]:
sets = tests.drop(tests[(tests.over <= 11.0) | (tests.over >=16.0)].index)
sets.reset_index(drop = True)
yt1 = sets['total_runs_y']
yt2 = sets['winner']
sets.drop(columns = ['id', 'date', 'total_runs_y', 'winner'], inplace = True)
sets.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12.0,5.0,5.0,1,11.1,12.0,5.0,1,0,0,95,2
1,12.0,5.0,5.0,1,11.2,12.0,5.0,1,0,0,95,2
2,12.0,5.0,5.0,1,11.3,12.0,5.0,1,0,0,96,2
3,12.0,5.0,5.0,1,11.4,12.0,5.0,1,0,0,96,2
4,12.0,5.0,5.0,1,11.5,12.0,5.0,1,0,0,97,2


In [114]:
yt1 = yt1.ravel()

In [115]:
xs = sc.fit_transform(sets)

In [116]:
predictions = model.predict(xs)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=yt1
sample.head(10)

,Predict,Actual
0,170.699493,159
1,169.386719,159
2,169.521774,159
3,168.239075,159
4,168.390411,159
5,168.558395,159
6,165.314148,159
7,168.277039,159
8,168.450699,159
9,168.624359,159


In [117]:
ar1 = sample['Actual'].unique()
results = pd.DataFrame()
for items in ar1:
  # print(items)
  try2 = sample[sample.Actual == items]
  result = (try2.sum())/try2.shape[0]
  # print(result)
  res = result.values.reshape(1,2)
  # print(res)
  results = results.append(pd.DataFrame(res))

In [118]:
results.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)
results.reset_index(drop = True)
print(custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))

37.142857142857146
